# Description of Search

Generate a list of every time in the Tanak there is a noun with a pronominal suffix functioning as a vocative...Special interest is on nouns within the semantic field of familial terms (father, mother, daughter, son, brother, sister). Also there is a special interest in the Torah passages.

The results are exported to a spreadsheet.

## Import ETCBC Database Processor

In [6]:
import csv

from tf.fabric import Fabric

TF = Fabric(modules='hebrew/etcbc4c')

This is Text-Fabric 2.3.6
Api reference : https://github.com/ETCBC/text-fabric/wiki/Api
Tutorial      : https://github.com/ETCBC/text-fabric/blob/master/docs/tutorial.ipynb
Data sources  : https://github.com/ETCBC/text-fabric-data
Data docs     : https://etcbc.github.io/text-fabric-data
Shebanq docs  : https://shebanq.ancient-data.org/text
Slack team    : https://shebanq.slack.com/signup
Questions? Ask shebanq@ancient-data.org for an invite to Slack
109 features found and 0 ignored


## Import Features from the ETCBC Text (version 4c)

### The following features are needed:

* `typ`- type, a feature stored on clauses. To find vocative clauses. [see all possible codes](https://etcbc.github.io/text-fabric-data/features/hebrew/etcbc4c/typ)
* `prs` - pronominal suffix, a feature stored on words. To find pronominal suffixes. [see description](https://etcbc.github.io/text-fabric-data/features/hebrew/etcbc4c/prs)
* `lex` - lexeme, a "feature" associated with words in ETCBC transcription. To find familial terms. [description](https://etcbc.github.io/text-fabric-data/features/hebrew/etcbc4c/lex)

### The following optional features will be added for additional comparisons [(see all features)](https://etcbc.github.io/text-fabric-data/features/hebrew/etcbc4c/0_overview.html):

* `function` (for phrases), `domain` (narrative, discursive, quotation)

In [7]:
# Features loaded here
# grammatical units stored on linguistic "objects" in etcbc
# we load the features associated with those objects
api = TF.load('''
              book chapter verse
              typ
              prs
              lex
              function domain gloss
              ''')

api.makeAvailableIn(globals())

  0.00s loading features ...
   |     0.01s B book                 from /Users/Cody/github/text-fabric-data/hebrew/etcbc4c
   |     0.01s B chapter              from /Users/Cody/github/text-fabric-data/hebrew/etcbc4c
   |     0.01s B verse                from /Users/Cody/github/text-fabric-data/hebrew/etcbc4c
   |     0.21s B typ                  from /Users/Cody/github/text-fabric-data/hebrew/etcbc4c
   |     0.13s B prs                  from /Users/Cody/github/text-fabric-data/hebrew/etcbc4c
   |     0.12s B lex                  from /Users/Cody/github/text-fabric-data/hebrew/etcbc4c
   |     0.08s B function             from /Users/Cody/github/text-fabric-data/hebrew/etcbc4c
   |     0.02s B domain               from /Users/Cody/github/text-fabric-data/hebrew/etcbc4c
   |     0.00s B gloss                from /Users/Cody/github/text-fabric-data/hebrew/etcbc4c
   |     0.00s Feature overview: 103 for nodes; 5 for edges; 1 configs; 7 computed
  5.67s All features loaded/computed - for

## Run the Search

In [8]:
# a list to store the results
results = []

# familial lexemes in ETCBC transcription
familial_lexemes = {'>B/','>M/','BT/',
                    'BN/','>X/','>XWT/'}

# iterate through all clauses in HB
for clause in F.otype.s('clause_atom'):
    
    # check for vocative
    if F.typ.v(clause) == 'Voct':
        
        # list of words with pronominal suffixes, if any
        suffixed_words = [word for word in L.d(clause, otype='word')
                             if F.prs.v(word) not in {'absent','n/a'} # check for pronominal
                         ]
        
        familial_words = [F.gloss.v(L.u(word, otype='lex')[0]) for word in suffixed_words
                             if F.lex.v(word) in familial_lexemes
                         ]

        # if there are suffixed words, save results
        if suffixed_words:
            
            # get data about the environment around the vocative
            verse = L.u(clause, otype='verse')[0]
            phrases = L.d(clause, otype='phrase')
            phrase_functions = tuple(F.function.v(phrase) for phrase in phrases)
            functional_clause = L.u(clause, otype='clause')[0]
            domain = F.domain.v(functional_clause)
            
            # format data into plain text
            passage_text = T.sectionFromNode(clause)
            vocative_text = T.text(L.d(clause, otype='word'))
            verse_text = T.text(L.d(verse, otype='word'))
            functions_text = ', '.join(phrase_functions)
            familial_words = ', '.join(familial_words)
            
            # consolidate data into dict
            result_data = {'Passage':passage_text,
                           'Familial?':familial_words,
                           'Vocative':vocative_text,
                           'Verse':verse_text,
                           'Domain':domain,
                           'Phrase Functions':functions_text}
            
            results.append(result_data)
            
print(f'Finished with {len(results)} results!')

Finished with 358 results!


## Move Familial Results to the Front For Sorted Results

In [9]:
sorted_results = []

for result in results:
    if result['Familial?']:
        sorted_results.append(result)
        
for result in results:
    if not result['Familial?']:
        sorted_results.append(result)
        
# double check we got everything
print(f'{len(sorted_results)} results sorted...')

358 results sorted...


## Export Both Sorted and Non-Sorted Results to Spreadsheet

In [10]:
fieldnames = ('Passage','Familial?','Vocative','Verse','Domain','Phrase Functions')

with open('familial_first.csv','w') as familial_results:
    writer = csv.DictWriter(familial_results, fieldnames=fieldnames)
    writer.writeheader()
    writer.writerows(sorted_results)
    
with open('all.csv','w') as all_results:
    writer = csv.DictWriter(all_results, fieldnames=fieldnames)
    writer.writeheader()
    writer.writerows(results)